In [122]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
from tqdm import tqdm, tqdm_notebook
tqdm.pandas()

In [123]:
users = pd.read_csv('./data/users.csv')
reviews = pd.read_csv('./data/reviews.csv', low_memory=False)
aspects = pd.read_csv('./data/aspects.csv', low_memory=False)
features = pd.read_csv('./data/features.csv', low_memory=False)
orgs = pd.read_csv('./data/organisations.csv', low_memory=False)
rubrics = pd.read_csv('./data/rubrics.csv', low_memory=False)

test = pd.read_csv('./data/test_users.csv')

* reviews - В этом файле дана информация об отзывах и оценках, оставленных некоторым множеством жителей Москвы и Санкт-Петерубрга в течение обучающего периода
* organisations - Информация об организациях
* users - Информация о городе проживания пользователя
* aspects - Описание извлекаемых из отзывов аспектов. Множество аспектов извлекается из отзыва с помощью NLP-алгоритма и может быть неточным.
* features - Описание особенностей организаций. Как правило, множество особенностей организации заполняется ее владельцем и может быть неточным.
* rubrics - Описание рубрик организаций
* test_users - Множество пользователей, для которых необходимо сделать предсказание

In [124]:
to_list = lambda rubrics: [int(rubric) for rubric in str(rubrics).split(' ')]
def apply_to_columns(df, columns, func=to_list):
    for column in columns:
        df.loc[~df[column].isnull(), column] = df.loc[~df[column].isnull(), column].apply(func)

In [125]:
users.head()

,user_id,city
0,523295021912509756,msk
1,11952159487361099606,msk
2,16879036589969590999,msk
3,12791716990148606332,msk
4,11642393216024958726,msk


In [126]:
orgs.head()

,org_id,city,average_bill,rating,rubrics_id,features_id
0,16848414477362211020,spb,1000.0,4.479702,30776 31375,1018 1509 11177 11617 11629 11704 11867 20422 ...
1,1430604733320164116,spb,1000.0,4.514509,30776 30770,246 1018 11617 11629 11704 11867 20422 21247 3...
2,9880309324224147401,spb,1000.0,3.884615,30770 30774,1018 11177 11617 11629 11704 11867 20422 21247...
3,5617879987171966456,spb,1000.0,NaN,30774 30775,1018 1509 10596 11177 11629 11634 11704 11867 ...
4,5241461680470612149,spb,1000.0,4.532468,30776,1018 11177 11617 11629 11704 11867 20422 21247...


In [127]:
cols = ['rubrics_id', 'features_id']
apply_to_columns(orgs, cols)
orgs.head()

,org_id,city,average_bill,rating,rubrics_id,features_id
0,16848414477362211020,spb,1000.0,4.479702,"[30776, 31375]","[1018, 1509, 11177, 11617, 11629, 11704, 11867..."
1,1430604733320164116,spb,1000.0,4.514509,"[30776, 30770]","[246, 1018, 11617, 11629, 11704, 11867, 20422,..."
2,9880309324224147401,spb,1000.0,3.884615,"[30770, 30774]","[1018, 11177, 11617, 11629, 11704, 11867, 2042..."
3,5617879987171966456,spb,1000.0,NaN,"[30774, 30775]","[1018, 1509, 10596, 11177, 11629, 11634, 11704..."
4,5241461680470612149,spb,1000.0,4.532468,[30776],"[1018, 11177, 11617, 11629, 11704, 11867, 2042..."


In [128]:
orgs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66405 entries, 0 to 66404
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   org_id        66405 non-null  uint64 
 1   city          66405 non-null  object 
 2   average_bill  31800 non-null  float64
 3   rating        53257 non-null  float64
 4   rubrics_id    66405 non-null  object 
 5   features_id   51702 non-null  object 
dtypes: float64(2), object(3), uint64(1)
memory usage: 3.0+ MB


In [129]:
reviews = reviews.merge(users, on='user_id')
reviews = reviews.rename({'city': 'user_city'}, axis=1)
reviews = reviews.merge(orgs[['org_id', 'city']], on='org_id')
reviews = reviews.rename({'city': 'org_city'}, axis=1)
reviews

,user_id,org_id,rating,ts,aspects,user_city,org_city
0,16998268288908323644,7184895086928047809,2.0,105,NaN,msk,msk
1,3121447338909258868,7184895086928047809,5.0,464,NaN,msk,msk
2,1970649778250883025,7184895086928047809,3.0,789,NaN,msk,msk
3,7554889464530643866,7184895086928047809,4.0,936,NaN,msk,msk
4,15907910894057053620,7184895086928047809,1.0,1143,NaN,msk,msk
...,...,...,...,...,...,...,...
3640830,16504916268155591133,11379950099553543298,1.0,1138,NaN,spb,spb
3640831,6729633349339708345,4127027708972853576,5.0,984,NaN,msk,msk
3640832,12811636719149152603,1870939193149876281,5.0,389,NaN,msk,msk
3640833,16479336894539955366,9457835296761142609,5.0,1068,NaN,msk,msk


In [130]:
columns = ['aspects']
apply_to_columns(reviews, columns)

In [131]:
reviews

,user_id,org_id,rating,ts,aspects,user_city,org_city
0,16998268288908323644,7184895086928047809,2.0,105,NaN,msk,msk
1,3121447338909258868,7184895086928047809,5.0,464,NaN,msk,msk
2,1970649778250883025,7184895086928047809,3.0,789,NaN,msk,msk
3,7554889464530643866,7184895086928047809,4.0,936,NaN,msk,msk
4,15907910894057053620,7184895086928047809,1.0,1143,NaN,msk,msk
...,...,...,...,...,...,...,...
3640830,16504916268155591133,11379950099553543298,1.0,1138,NaN,spb,spb
3640831,6729633349339708345,4127027708972853576,5.0,984,NaN,msk,msk
3640832,12811636719149152603,1870939193149876281,5.0,389,NaN,msk,msk
3640833,16479336894539955366,9457835296761142609,5.0,1068,NaN,msk,msk


In [132]:
reviews = reviews.merge(orgs, on='org_id', how='left')\
    .rename(columns={'rating_x': 'rating', 'rating_y':'avg_rating'})\
    .drop(['city', 'ts'], axis=1)

In [133]:
reviews['features_id'] = reviews['features_id'].fillna(0)

## Для каждого пользователя найдем особенность, которая чаще всего встречается в заведениях где оценка больше 4

In [134]:
def fav_feature(reviews_df, users_df):
    pass

In [135]:
# набор отзывов только от туристов
tourist_reviews = reviews[reviews['rating'] >= 4.0]
tourist_reviews = tourist_reviews[tourist_reviews['user_city'] != tourist_reviews['org_city']]

In [136]:
# выбираем самые популярные места среди туристов из Москвы и Питера
msk_orgs = tourist_reviews[tourist_reviews['org_city'] == 'msk']['org_id']
msk_orgs = msk_orgs.value_counts().index[:20].to_list()

spb_orgs = tourist_reviews[tourist_reviews['org_city'] == 'spb']['org_id']
spb_orgs = spb_orgs.value_counts().index[:20].to_list()


msk_orgs = str(' '.join(map(str, msk_orgs)))
spb_orgs = str(' '.join(map(str, spb_orgs)))

In [137]:
test_users = pd.read_csv('data/test_users.csv')
test_users['city'] = test_users.merge(users, on='user_id')['city']

In [138]:
choose = lambda x: spb_orgs if x['city'] == 'msk' else msk_orgs
target = test_users.apply(choose, axis=1)

predictions = test_users[['user_id']]
predictions['target'] = target

predictions.head()

,user_id,target
0,3545210947248911048,12046097390037935713 5002407858008059043 14814...
1,15271987121288045390,12046097390037935713 5002407858008059043 14814...
2,15016858616184265932,12046097390037935713 5002407858008059043 14814...
3,12457244142928722989,12046097390037935713 5002407858008059043 14814...
4,13339684649926251468,15250345250621165867 13573322486152844808 9104...
